# Uncertainty Quantification in Deep Learning (Regression)

Rubén Martínez Cantín, Javier Civera

rmcantin@unizar.es

University of Zaragoza



In this part, we will analyze the performance of scalable Bayesian networks for regression problems. Specifically, we will use the Boston dataset

Do the following tasks:
* Run the code and make sure you understand all of the cells.
* Train a neural network (e.g., the one defined in the notebook) and report some regression metrics (e.g., mean absolute error)
* Train a Bayesian neural network with aleatoric uncertainty and report metrics for the error and the uncertainty. For the uncertainty, note that you have a function `ause(abs error,y var)` in the Colab notebook.
* Train a Bayesian neural network with aleatoric and epistemic uncertainty, the latest using an ensemble of N networks. Report error metrics and AUSE and its dependence with the number of models in the ensemble. The global uncertainty, combining epistemic and aleatoric, can be approximated by:

$$  \sigma^2 = \underbrace{\frac{1}{M} \sum_{i=1}^M \left(y_i - \bar{y}\right)^2}_{\text{epistemic}} + \underbrace{\frac{1}{M} \sum_{i=1}^M \sigma_{y,i}^2}_{\text{aleatoric}}$$

where $\bar{y} = \frac{1}{M} \sum_{i=1}^M y_i$ and $(y_i, \sigma_{y,i}^2)$
correspond to the mean and the variance predicted by the i-th network of the
ensemble.

* Train a Bayesian neural network with aleatoric and epistemic uncertainty, this time using MC Dropout. For that, add Dropout layers to your model using:
```
x = F.Dropout(x, p=dropout_p, training=True)
```
instead of
```
x = self.drop(x)
```
The difference is that `self.drop` use a `nn.Module` which already defines if the network is training or testing (and deactivates dropout for testing). Using the functional version `F.Dropout` we can force the layer to be always active by tricking the layer to think to it is always training.

Analyze its performance, the dependence of the errors and uncertainty calibration with the number of forward passes, and compare it against ensembles.

* Are the test errors consistent with the predicted uncertainties? The estimation of the uncertainty is under- or over-confident?
* Generate an out-of-distribution test sample and check its high predicted uncertainty.

* Run the code for Laplace approximation with different configurationas and evaluate the performance.

If you want, you can make several copies of this notebook and train each method in a different colab (Monte Carlo, Ensembles and Laplace).

In [ ]:
#@title Install Dependencies
#@markdown This is only required for laplace. You can remove this for MC-dropout and ensembles
!pip install laplace-torch

In [ ]:
#@title Import Dependencies
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dists
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#@markdown Again, the last import is only required for Laplace. Remove it if you are only doing MC or ensembles.
from laplace import Laplace

In [ ]:
#@title Define Hyperparameters
#@markdown Hyperparameters for the different algoritms, model, etc. Feel free to add more if needed.
#@markdown
#@markdown **TODO:** The hyperparameters are NOT optimal. Change them as need it.
#@markdown Also, consider reducing the number of epochs and ensembles for development.


num_ensembles = 32 #@param {type: "integer"}
num_epochs = 200  #@param {type: "integer"} # number of times which the entire dataset is passed throughout the model
batch_size = 100 #@param {type: "integer"} # the size of input data took for one iteration
lr = 1e-4 #@param {type: "number"} # size of step
weight_decay = 0.00005 #@param {type: "number"}
dropout_p = 0.5 #@param {type: "number"}

n_epochs_Laplace = 10 #@param {type: "integer"} # epochs for Laplace hyperparameters

use_gpu = True #@param {type: "boolean"}
gpu_id = 0 #@param {type: "integer"}

if torch.cuda.is_available() and use_gpu:
    device = torch.device("cuda:" + str(gpu_id))
    print("Using GPU id {}".format(gpu_id))
else:
    device = torch.device("cpu")
    print("GPU not detected. Defaulting to CPU.")

Using GPU id 0


# Loading the dataset

In the next block, we are going to import the California housing dataset from sklearn. If you want, you can use any other regression datasets. For example, you can try any of the following datasets from UCI:

* Wine quality dataset:
https://archive-beta.ics.uci.edu/dataset/186/wine+quality
```
import pandas as pd
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv" --no-check-certificate
data = pd.read_csv('winequality-red.csv', header=1, delimiter=';').values
data = data[np.random.permutation(np.arange(len(data)))]
```

* Concrete compression:
https://archive-beta.ics.uci.edu/dataset/165/concrete+compressive+strength
```
import pandas as pd
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls" --no-check-certificate
data = pd.read_excel('Concrete_Data.xls', header=0, delimiter="\s+").values
data = data[np.random.permutation(np.arange(len(data)))]
```

* Combined cycle power plant:
https://archive-beta.ics.uci.edu/dataset/294/combined+cycle+power+plant
```
import pandas as pd
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip" --no-check-certificate
zipped = zipfile.ZipFile("CCPP.zip")
data = pd.read_excel(zipped.open('CCPP/Folds5x2_pp.xlsx'), header=0, delimiter="\t").values
```

* Yacht hydrodynamics: https://archive-beta.ics.uci.edu/dataset/243/yacht+hydrodynamics
```
import pandas as pd
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data" --no-check-certificate
data = pd.read_csv('yacht_hydrodynamics.data', header=1, delimiter='\s+').values
data = data[np.random.permutation(np.arange(len(data)))]
```


**Important:** After loading any of the previous datasets, you should include the following lines:
```
in_dim = data.shape[1] - 1

X = torch.Tensor(data[:, :in_dim])
Y = torch.Tensor(data[:, in_dim:])
```

In [ ]:
#@title Loading California housing.
#@markdown Remove if you load other dataset.
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

X = torch.Tensor(data.data)
Y = torch.Tensor(data.target)

In [ ]:
#@title Data preprocessing
#@markdown We are going to build the train/test split and preprocess all the data
#@markdown (normalization, building data loaders, etc.)

import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler

#import sklearn model selection package to split data into train and test models
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state = 42)

# We rescale de data to avoid numerical problems.
# The scaling factor is computed only with training data to avoid transfering
# knowledge to test data, but applied in both datasets.
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

train_dataset = data_utils.TensorDataset(torch.Tensor(x_train),torch.Tensor(y_train))
train_gen = data_utils.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

test_dataset = data_utils.TensorDataset(torch.Tensor(x_test),torch.Tensor(y_test))
test_gen = data_utils.DataLoader(test_dataset, batch_size=batch_size, shuffle = False)

In [ ]:
#@title Define model class
#@markdown We have defined two models. One traditional regression model where the
#@markdown output is predicted value (assuming homoscedastic noise) and other with
#@markdown two-heads where we both predict the value and the aleatoric uncertainty
#@markdown (heteroscedastic noise).
#@markdown
#@markdown **TODO:** This model is not optimal for any of the datasets. You should try different models.


input_size = x_train.shape[1]

class HomoMLP(nn.Module):
  def __init__(self, input_size):
    super(HomoMLP,self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 1)
    self.relu = nn.ReLU()

  def forward(self,x):
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    return self.fc4(x)

class HeteroMLP(HomoMLP):
  def __init__(self, input_size):
    super(HeteroMLP,self).__init__(input_size)
    self.fc4 = nn.Linear(32, 2)

# Metrics

In the following section, we have defined some metrics for training or evaluation that you can use:

* **Negative log likelihood (NLL)**: Asumes `y_pred[:,0]` is the mean $\mu$ and `y_pred[:,1]` is the log of the variance $\log \sigma^2$; `y_true` is the ground truth values.  You can also use

```
-dists.Normal(y_pred[:,0], torch.exp(y_pred[:,1])).log_prob(y_true).mean()
```

* **Mean absolute error (MAE)**: Asumes `y_pred[:,0]` is the mean $\mu$ and `y_pred[:,1]` is the log of the variance $\log \sigma^2$; `y_true` is the ground truth values.  You can also use:

```
mae = nn.L1Loss()
mae(y_pred[:,0], y_true)
```

* **AUSE**: Computes the ause given a vector/array with the absolute errors and the variances. The arrays can be numpy arrays or pytorch tensors. It also **plots** the figure with the sparsification curves. Feel free to modify the plot to your preferences.

* **Root mean square error (RMSE)**: That function is not implemented, but you can easily do it based on MAE. You can also use the MSE loss from pytorch:

```
mse = nn.MSELoss()
torch.sqrt(mse(y_pred[:,0], y_true))
```


In [ ]:
#@title Metrics and losses

def neg_log_likelihood(y_pred, y_true):
  '''Computes the negative log-likelihood for training
  '''
  y_mean = y_pred[:,0]
  y_logvar = y_pred[:,1]
  return torch.mean(0.5*torch.square(y_true - y_mean)/torch.exp(y_logvar) + 0.5 * y_logvar)

def mean_abs_error(y_pred, y_true):
  '''Outputs the mae, for inspecting the training
  '''
  y_mean = y_pred[:,0]
  return torch.mean(torch.abs(y_true - y_mean))


def ause(abs_error, y_var, plot_curves=True):
  '''Computes the AUSE error based on the variance and absolute error.
     It also plots the figure with the sparsification curves.
     Feel free to modify the plot to your preferences.
  '''
  if torch.is_tensor(abs_error):
    abs_error = abs_error.detach().cpu().numpy()
  if torch.is_tensor(y_var):
    y_var = y_var.detach().cpu().numpy()

  abs_error /= np.sum(abs_error)
  idx_errors = np.argsort(abs_error)[::-1]
  idx_variances = np.argsort(y_var)[::-1]
  scurve_errors = np.array([1])
  scurve_variances = np.array([1])
  abs_error_sorted = abs_error[idx_errors]
  variances_sorted = abs_error[idx_variances]

  for _ in range(len(abs_error)-1):
    abs_error_sorted = np.delete(abs_error_sorted,0)
    scurve_errors = np.append(scurve_errors,np.sum(abs_error_sorted))
    variances_sorted = np.delete(variances_sorted,0)
    scurve_variances = np.append(scurve_variances,np.sum(variances_sorted))

  percentages = (np.arange(len(scurve_errors))+1) / len(scurve_errors)

  if plot_curves:
    plt.plot(percentages, scurve_errors)
    plt.plot(percentages, scurve_variances)
    plt.title('Sparsification error curves')
    plt.ylabel('mae')
    plt.xlabel('ratio of samples removed')
    plt.legend(['oracle', 'model'], loc='lower left')
    plt.show()

  integral_errors = np.trapz(scurve_errors, percentages)
  integral_variances = np.trapz(scurve_variances, percentages)

  return np.abs(integral_errors - integral_variances)

In [ ]:
#@title Train and evaluate model
#@markdown Functions for training and evaluating the model. Note that the training
#@markdown function is set to evaluate the model after each epoch. You can change
#@markdown that behaviour.

def eval_step(model, test_gen, loss_function):
    model.eval()
    train_acc = 0
    val_loss = 0
    with torch.no_grad():
      for i ,(inputs,labels) in enumerate(test_gen):
        inputs = inputs.to(device)

        # WARNING: Depending on the dataset, you might need to squeeze
        # or unsqueeze the labels
        labels = labels.to(device).unsqueeze(1)

        outputs = model(inputs)
        loss_val = loss_function(outputs, labels)

        # Calculate and accumulate accuracy metric across all batches
        train_acc += mean_abs_error(outputs, labels)

    val_loss = loss_val.item()
    return val_loss, train_acc/len(test_gen)

def train_model(model, train_gen, test_gen, loss_function, optimizer):
    for epoch in range(num_epochs):
        model.train()
        for i ,(inputs,labels) in enumerate(train_gen):
            inputs = inputs.to(device)
            # WARNING: Depending on the dataset, you might need to squeeze
            # or unsqueeze the labels
            labels = labels.to(device).unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

        val_loss, train_acc = eval_step(model, test_gen, loss_function)
        mae = mean_abs_error(outputs, labels)
        print('Epoch [%d/%d], Loss: %.4f, MAE: %.4f, Val loss: %.4f, Val MAE: %.4f'
                      %(epoch+1, num_epochs, loss.item(), mae, val_loss, train_acc))

In [ ]:
#@title Training a base model (homoscedastic)
#@markdown This a pure deterministic model, assuming homoscedastic noise.

homo_model = HomoMLP(input_size).to(device)
loss_function = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(homo_model.parameters(), lr=lr, weight_decay=weight_decay)
train_model(homo_model, train_gen, test_gen, loss_function, optimizer)

In [ ]:
#@title Training a heteroscedastic model
#@markdown This a pure deterministic model, assuming heteroscedastic noise.

hetero_model = HeteroMLP(input_size).to(device)
loss_function = neg_log_likelihood
optimizer = torch.optim.Adam(hetero_model.parameters(), lr=lr, weight_decay=weight_decay)
train_model(hetero_model, train_gen, test_gen, loss_function, optimizer)

In [ ]:
#@title Training deep ensembles (with heteroscedastic noise)

#@markdown This is the same code as before for deep ensembles.
#@markdown
#@markdown Because all models inherit from `nn.Module`, pytorch already perform
#@markdown random initialization for us each time we create a new model.
#@markdown You can train ensembles with different base models (homoscedastic,
#@markdown different architectures, etc.)

models = []

for im in range(num_ensembles):
    print('Training model %d out of %d...........................................................' % (im+1, num_ensembles))
    loss_function = neg_log_likelihood
    hetero_model = HeteroMLP(input_size).to(device)
    optimizer = torch.optim.Adam(hetero_model.parameters(), lr=lr, weight_decay=weight_decay)
    train_model(hetero_model, train_gen, test_gen, loss_function, optimizer)
    models.append(hetero_model)

# Laplace model

In this part of the lab, we are going to use the Laplace approximation to approximate the posterior and predictive posterior distributions.

For the implemenation, we are going to use the Laplace-redux library
* Code: https://github.com/AlexImmer/Laplace
* Paper: https://arxiv.org/abs/2106.14806
* Docs: https://aleximmer.github.io/Laplace/

We assume that we have already a pretrained model. For simplicity, we are going to use the **homoscedastic model** and optimize a single **noise variance** value separately for all the inputs $\sigma_n^2$. Furthermore, we will also optimize the prior distribution, which we assume to be in ther form $\mathcal{N}(0, \lambda I)$, where $\lambda$ is called the **prior precision**.

You can play with the different configurations of weights and Hessian structure. See fig. 2 in the paper for a quick overview of the options. You can try using a less expensive approximation and optimize the hyperparamenters for more epochs.

**Note:** The hyperparameters for noise variance and prior precision are optimized in log-space $(\log \sigma_n^2, \log \lambda)$ because it is numerically more stable and produce better results.

We provide some code to evaluate the Laplace model as an example, but you should use your own metrics and analysis.

In [ ]:
#@title Fitting Laplace model
#@markdown We are going to use a pretrained model.
#@markdown See figure 2 of the paper on how to configure the Laplace approximation.
#@markdown Try changing the parameters of the `Laplace(...)` model (weights, Hessian...)

# Configure and fit the model
print('Fitting post-hoc Laplace model....................')
la = Laplace(
    homo_model,
    'regression',
    subset_of_weights='all',
    hessian_structure='full'
    )

la.fit(train_gen)
print('Done')

log_prior, log_sigma = torch.ones(1, requires_grad=True), torch.ones(1, requires_grad=True)
hyper_optimizer = torch.optim.Adam([log_prior, log_sigma], lr=1e-1)
for i in range(n_epochs_Laplace):
    hyper_optimizer.zero_grad()
    neg_marglik = - la.log_marginal_likelihood(log_prior.exp(), log_sigma.exp())
    neg_marglik.backward()
    hyper_optimizer.step()

    print('Epoch [%d/%d], Prior: %.4f, Sigma: %.4f'
                      %(i+1, n_epochs_Laplace, log_prior.exp(), log_sigma.exp()))

In [ ]:
#@title Evaluate Laplace model
#@markdown We compare the Laplace model to the homoscedastic and heteroscedastic
#@markdown deterministic models.
#@markdown We show some metrics as an example. You should do your own analysis.

def predict(dataloader, model, bayes=None):
    py = []
    pstd = []
    pygt = []

    for x, y_real in dataloader:
        if bayes == 'Laplace':
            mu, var = model(x.to(device), pred_type='glm')
            y_pred = mu.squeeze().detach()
            y_var_epistemic = var.squeeze().detach()
            y_var_aleatoric = model.sigma_noise.item() ** 2
            y_pred_std = torch.sqrt(y_var_epistemic + y_var_aleatoric)
            pstd.append(y_pred_std)
        elif bayes == 'Heteroscedastic':
            output = model(x.to(device))
            y_pred = output[:,0].squeeze().detach()
            y_pred_std = torch.sqrt(torch.exp(output[:,1])).squeeze().detach()
            pstd.append(y_pred_std)
        else:
            y_pred = model(x.to(device)).squeeze().detach()

        pygt.append(y_real)
        py.append(y_pred)

    if bayes is not None:
        return torch.cat(pygt).cpu(), torch.cat(py).cpu(), torch.cat(pstd).cpu()
    else:
        return torch.cat(pygt).cpu(), torch.cat(py).cpu()

# Use torch.no_grad to call the losses as pure metrics without worrying about computing extra gradients for backpropagation.
with torch.no_grad():
    y_test, y_predMAP = predict(test_gen, homo_model)
    y_test, y_predHetero, y_pred_stdHetero = predict(test_gen, hetero_model, bayes='Heteroscedastic')
    y_test, y_pred, y_pred_std = predict(test_gen, la, bayes='Laplace')


    mse = nn.MSELoss()
    mae = nn.L1Loss()
    print(f'[MAP] MSE: {mse(y_predMAP, y_test):.3}, MAE: {mae(y_predMAP, y_test):.3}')

    nll = -dists.Normal(y_predHetero, y_pred_stdHetero).log_prob(y_test).mean()
    print(f'[LAP] NLL: {nll:.3}, MSE: {mse(y_pred, y_test):.3}, MAE: {mae(y_pred, y_test):.3}')

    nll = -dists.Normal(y_pred, y_pred_std).log_prob(y_test).mean()
    print(f'[LAP] NLL: {nll:.3}, MSE: {mse(y_pred, y_test):.3}, MAE: {mae(y_pred, y_test):.3}')


In [ ]:
#Histogram of the distribution of prectictions with respect to the true value
fig,ax = plt.subplots(3,1)
ax[0].hist(y_predMAP-y_test, bins=50, range=(-5,5))
ax[1].hist(y_predHetero-y_test, bins=50,range=(-5,5))
ax[2].hist(y_pred-y_test, bins=50,range=(-5,5))

In [ ]:
#AUSE based on the mean absolute error (MAE). You can also use the MSE o RMSE instead of the MAE.
abs_error = torch.abs(y_pred-y_test).detach().cpu().numpy()
y_var = (y_pred_std**2).detach().cpu().numpy()
print("AUSE:", ause(abs_error, y_var))